In [7]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import os




In [3]:
df = pd.read_csv('canlii_final_report_20.csv')
df.columns
df.head()

,Judge,Heard_Date,Released_Date,Case_Title,URL
0,C.A. Brannagan,27 November 2025,2025-12-31,R. v. M.T.,https://www.canlii.org/en/on/oncj/doc/2025/202...
1,Fergus ODonnell,Unknown,2025-12-30,R. v. J.G.,https://www.canlii.org/en/on/oncj/doc/2025/202...
2,S. Robichaud,"October 27 to October 31, November 12, 17-25, ...",2025-12-29,R. v. Laguerre,https://www.canlii.org/en/on/oncj/doc/2025/202...
3,S. Robichaud,"December 29, 2025",2025-12-29,R. v. Khosa,https://www.canlii.org/en/on/oncj/doc/2025/202...
4,S. G. Pratt,"4 November, 17 December, 2025",2025-12-24,R. v. Lachance,https://www.canlii.org/en/on/oncj/doc/2025/202...


In [13]:
import undetected_chromedriver as uc
import time
import os
import pandas as pd
from bs4 import BeautifulSoup

def setup_undetected_driver():
    """Set up undetected Chrome driver with auto-update"""
    options = uc.ChromeOptions()
    
    # Performance settings
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    # Create driver with version detection
    driver = uc.Chrome(
        options=options,
        use_subprocess=True,
        version_main=None  # Auto-detect Chrome version
    )
    
    return driver

def fetch_with_undetected(driver, url, wait_time=5):
    """Fetch webpage content"""
    try:
        driver.get(url)
        time.sleep(wait_time)
        
        # Check if we got CAPTCHA'd
        if "captcha-delivery.com" in driver.page_source:
            print("   ⚠️  DataDome CAPTCHA detected - waiting...")
            time.sleep(10)
        
        return driver.page_source
        
    except Exception as e:
        print(f"   ❌ Error: {e}")
        return None

def extract_case_text(html_content):
    """Extract clean text from CanLII HTML"""
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Try to find the main case content
    case_content = soup.find('div', {'class': 'documentContent'})
    
    if case_content:
        return case_content.get_text(separator='\n', strip=True)
    else:
        return soup.get_text(separator='\n', strip=True)

def scrape_canlii_cases(df):
    """Scrape all cases with undetected-chromedriver"""
    
    os.makedirs('court_case_texts', exist_ok=True)
    
    print("🚀 Setting up undetected Chrome driver...")
    driver = setup_undetected_driver()
    
    successful = 0
    failed = 0
    
    try:
        for idx, url in enumerate(df['URL']):
            if pd.notna(url):
                print(f"\n[{idx+1}/{len(df)}] Fetching: {url}")
                
                html_content = fetch_with_undetected(driver, url, wait_time=5)
                
                if html_content and "captcha-delivery.com" not in html_content:
                    clean_text = extract_case_text(html_content)
                    
                    filepath = f'court_case_texts/case_{idx}.txt'
                    with open(filepath, 'w', encoding='utf-8') as f:
                        f.write(clean_text)
                    
                    successful += 1
                    print(f"   ✅ Saved ({len(clean_text)} chars)")
                else:
                    failed += 1
                    print(f"   ❌ Failed (CAPTCHA or error)")
                
                # Random delay
                import random
                delay = random.uniform(3, 6)
                print(f"   ⏳ Waiting {delay:.1f}s...")
                time.sleep(delay)
        
    finally:
        print("\n🛑 Closing browser...")
        driver.quit()
    
    print(f"\n📊 Results: {successful} successful, {failed} failed")
    
    df['text_file_path'] = df.index.map(
        lambda idx: f'court_case_texts/case_{idx}.txt' 
        if os.path.exists(f'court_case_texts/case_{idx}.txt')
        else None
    )
    
    return df

# Run it
df = scrape_canlii_cases(df)
df.to_csv('cases_with_text.csv', index=False)

🚀 Setting up undetected Chrome driver...


SessionNotCreatedException: Message: session not created: cannot connect to chrome at 127.0.0.1:56921
from session not created: This version of ChromeDriver only supports Chrome version 145
Current browser version is 144.0.7559.133; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#sessionnotcreatedexception
Stacktrace:
0   undetected_chromedriver             0x00000001026db3b8 undetected_chromedriver + 6730680
1   undetected_chromedriver             0x00000001026d2bea undetected_chromedriver + 6695914
2   undetected_chromedriver             0x00000001020decf5 undetected_chromedriver + 453877
3   undetected_chromedriver             0x0000000102122639 undetected_chromedriver + 730681
4   undetected_chromedriver             0x0000000102121451 undetected_chromedriver + 726097
5   undetected_chromedriver             0x00000001021160a9 undetected_chromedriver + 680105
6   undetected_chromedriver             0x0000000102168

In [14]:
df.shape

(510, 5)